<a href="https://colab.research.google.com/github/mqanaq/BA820-B1-Team13/blob/main/Suji_Kim_BA820_M4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Develped - Q4: Red-flag Cases and Boundary Conditions**

**Project Milestone 4**

**Team:** B1 Team 13

**Team Member:** Suji Kim

# Introduction

# Executive Summary

To overcome the limitations of TF-IDF, we introduced SBERT, compared and analyzed three models other than Isolation Forest, and statistically verified the correlation with Venue.

# Setting Up Environment

In [21]:
# Core
import os
import re
import math
import json
import time
import string
import random
import warnings
from pathlib import Path
from collections import Counter, defaultdict

import warnings
import logging
os.environ['PYTHONWARNINGS'] = 'ignore'
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", message=".*datetime.datetime.utcnow().*")
logging.getLogger('jupyter_client').setLevel(logging.ERROR)

# Data
import numpy as np
import pandas as pd
from scipy.stats import chi2_contingency #add

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.io as pio
pio.renderers.default = 'colab' #add

# NLP / Text processing
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sentence_transformers import SentenceTransformer #add

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, PCA, TruncatedSVD
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.metrics import silhouette_score
from sklearn.metrics.pairwise import cosine_similarity

# Modeling / Anomaly detection
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor #add
from sklearn.svm import OneClassSVM #add

# Preprocessing
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# Association rules
from mlxtend.frequent_patterns import apriori, association_rules

# Dimensionality reduction for viz
from sklearn.manifold import TSNE
from umap import UMAP #add

# Data Importing, Inspection and Preparation

## Data Importing

In [22]:
fair_use_findings = pd.read_csv('https://raw.githubusercontent.com/rfordatascience/tidytuesday/main/data/2023/2023-08-29/fair_use_findings.csv')

## Data Inspection

The `fair_use_findings` table contains complementary case-level text, including summaries of key facts, legal issues, holdings, and descriptive tags. Inspection centers on text completeness and variability, as these fields support later analysis of language patterns, similarity, and thematic structure across cases.

| variable    | class     | description                                                                            |
| ----------- | --------- | -------------------------------------------------------------------------------------- |
| title       | character | The title of the case.                                                                 |
| case_number | character | The case number or numbers of the case.                                                |
| year        | character | The year in which the finding was made (or findings were made).                        |
| court       | character | The court or courts involved.                                                          |
| key_facts   | character | The key facts of the case.                                                             |
| issue       | character | A brief description of the fair use issue.                                             |
| holding     | character | The decision of the court in paragraph form.                                           |
| tags        | character | Comma- or semicolon-separated tags for this case.                                      |
| outcome     | character | A brief description of the outcome of the case. These fields have not been normalized. |

In [23]:
print("Dataset Info:")
print(fair_use_findings.info())

print("\nFirst 5 rows:")
print(fair_use_findings.head())

print("\nMissing Values:")
print(fair_use_findings.isnull().sum())

Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 251 entries, 0 to 250
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        251 non-null    object
 1   case_number  251 non-null    object
 2   year         251 non-null    object
 3   court        251 non-null    object
 4   key_facts    251 non-null    object
 5   issue        251 non-null    object
 6   holding      251 non-null    object
 7   tags         251 non-null    object
 8   outcome      251 non-null    object
dtypes: object(9)
memory usage: 17.8+ KB
None

First 5 rows:
                                               title  \
0                              De Fontbrune v. Wofsy   
1                          Sedlik v. Von Drachenberg   
2  Sketchworks Indus. Strength Comedy, Inc. v. Ja...   
3  Am. Soc'y for Testing & Materials v. Public.Re...   
4                           Yang v. Mic Network Inc.   

                                 

## Preparing Data

### Outcome Flag Construction

The outcome column is converted into a simple label for analysis. The text is cleaned and then grouped into three outcomes: fair use found, fair use not found, and indeterminate (preliminary, mixed, remand, or unclear). A binary `fair_use_found` flag is created only for the final outcomes, and indeterminate cases are left out of binary rate calculations.

In [24]:
# Count outcome column from fair_use_findings and reset index
outcome_counts = fair_use_findings["outcome"].astype(str).str.lower().str.strip().value_counts().reset_index()
fair_use_findings["outcome"] = fair_use_findings["outcome"].astype(str).str.lower().str.strip()
outcome_counts.columns = ["outcome", "count"]

# Display the counts
print(outcome_counts)

                                              outcome  count
0                                  fair use not found    100
1                                      fair use found     98
2         preliminary ruling, mixed result, or remand     28
3             preliminary finding; fair use not found      4
4                                        mixed result      3
5              preliminary ruling, fair use not found      3
6              fair use not found, preliminary ruling      3
7              preliminary ruling; fair use not found      2
8              fair use not found; preliminary ruling      2
9                          preliminary ruling, remand      1
10                                fair use not found.      1
11                                    fair use found.      1
12  preliminary ruling, fair use not found, mixed ...      1
13                 preliminary ruling, fair use found      1
14  fair use found; second circuit affirmed on app...      1
15                      

Based on the grouped outcome counts, outcomes fall into three clear categories. Entries labeled “Fair use found” (including minor punctuation or appeal notes) are treated as fair use found, and entries labeled “Fair use not found” (including punctuation variants) are treated as fair use not found. All remaining outcomes, such as preliminary rulings, mixed results, remands, and irregular text entries, are treated as indeterminate. A binary fair_use_found flag is then defined only for the final outcomes, while indeterminate cases are excluded from binary rate calculations.

In [25]:
outcome_map = {
    # FINAL: fair use found
    "fair use found": "FAIR_USE_FOUND",
    "fair use found.": "FAIR_USE_FOUND",
    "fair use found; second circuit affirmed on appeal.": "FAIR_USE_FOUND",

    # FINAL: fair use not found
    "fair use not found": "FAIR_USE_NOT_FOUND",
    "fair use not found.": "FAIR_USE_NOT_FOUND",

    # INDETERMINATE
    "preliminary ruling, mixed result, or remand": "INDETERMINATE",
    "preliminary finding; fair use not found": "INDETERMINATE",
    "mixed result": "INDETERMINATE",
    "preliminary ruling, fair use not found": "INDETERMINATE",
    "fair use not found, preliminary ruling": "INDETERMINATE",
    "preliminary ruling; fair use not found": "INDETERMINATE",
    "fair use not found; preliminary ruling": "INDETERMINATE",
    "preliminary ruling, remand": "INDETERMINATE",
    "preliminary ruling, fair use not found, mixed result": "INDETERMINATE",
    "preliminary ruling, fair use found": "INDETERMINATE",
    "fair use found; mixed result": "INDETERMINATE",
    "plaintiff patrick cariou published yes rasta, a book of portraits and landscape photographs taken in jamaica. defendant richard prince was an appropriation artist who altered and incorporated several of plaintiff’s photographs into a series of paintings and collages called canal zone that was exhibited at a gallery and in the gallery’s exhibition catalog. plaintiff filed an infringement claim, and the district court ruled in his favor, stating that to qualify as fair use, a secondary work must “comment on, relate to the historical context of, or critically refer back to the original works.” defendant appealed.": "INDETERMINATE",
}

In [26]:
# Replace outcome column values with the mapping in outcome_map
fair_use_findings["outcome"] = fair_use_findings["outcome"].replace(outcome_map)
fair_use_findings["outcome"].value_counts().reset_index()

,outcome,count
0,FAIR_USE_NOT_FOUND,101
1,FAIR_USE_FOUND,100
2,INDETERMINATE,50


### Column Cleaning Steps

The `year` column is converted to a numeric integer format to ensure it can be used reliably in grouping, filtering, and any downstream modeling steps. Any non-numeric or missing values are handled safely during conversion.

In [27]:
# Turn the year column to integer
fair_use_findings["year"] = pd.to_numeric(fair_use_findings["year"], errors="coerce").astype("Int64")

In [28]:
fair_use_findings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 251 entries, 0 to 250
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        251 non-null    object
 1   case_number  251 non-null    object
 2   year         250 non-null    Int64 
 3   court        251 non-null    object
 4   key_facts    251 non-null    object
 5   issue        251 non-null    object
 6   holding      251 non-null    object
 7   tags         251 non-null    object
 8   outcome      251 non-null    object
dtypes: Int64(1), object(8)
memory usage: 18.0+ KB


In [29]:
fair_use_findings.head()

,title,case_number,year,court,key_facts,issue,holding,tags,outcome
0,De Fontbrune v. Wofsy,39 F.4th 1214 (9th Cir. 2022),2022,United States Court of Appeals for the Ninth C...,Plaintiffs own the rights to a catalogue compr...,Whether reproduction of photographs documentin...,"The panel held that the first factor, the purp...",Education/Scholarship/Research; Photograph,FAIR_USE_NOT_FOUND
1,Sedlik v. Von Drachenberg,"No. CV 21-1102, 2022 WL 2784818 (C.D. Cal. May...",2022,United States District Court for the Southern ...,Plaintiff Jeffrey Sedlik is a photographer who...,Whether use of a photograph as the reference i...,"Considering the first fair use factor, the pur...",Painting/Drawing/Graphic; Photograph,INDETERMINATE
2,"Sketchworks Indus. Strength Comedy, Inc. v. Ja...","No. 19-CV-7470-LTS-VF, 2022 U.S. Dist. LEXIS 8...",2022,United States District Court for the Southern ...,Plaintiff Sketchworks Industrial Strength Come...,"Whether the use of protected elements, includi...","The court found that the first factor, the pur...",Film/Audiovisual; Music; Parody/Satire; Review...,FAIR_USE_FOUND
3,Am. Soc'y for Testing & Materials v. Public.Re...,"No. 13-cv-1215 (TSC), 2022 U.S. Dist. LEXIS 60...",2022,United States District Court for the District ...,"Defendant Public.Resource.Org, Inc., a non-pro...",Whether it is fair use to make available onlin...,"As directed by the court of appeals, the distr...",Education/Scholarship/Research; Textual Work; ...,INDETERMINATE
4,Yang v. Mic Network Inc.,"Nos. 20-4097-cv(L), 20-4201-cv (XAP), 2022 U.S...",2022,United States Court of Appeals for the Second ...,Plaintiff Stephen Yang (“Yang”) licensed a pho...,"Whether using a screenshot from an article, in...","On appeal, the court decided that the first fa...",News Reporting; Photography,FAIR_USE_FOUND


### Metadata Extraction & Text Preprocessing

In [30]:
# Metadata features
fair_use_findings['holding_word_count'] = fair_use_findings['holding'].fillna("").apply(lambda x: len(str(x).split()))
fair_use_findings['tag_count'] = fair_use_findings['tags'].fillna("").apply(lambda x: len(re.split(r'[;,]', str(x))) if x else 0)
median_year = fair_use_findings['year'].median()
fair_use_findings['year_filled'] = fair_use_findings['year'].fillna(median_year)

# Combine key facts + issue into one text field
fair_use_findings["text"] = (
    fair_use_findings["key_facts"].fillna("").astype(str) + " " +
    fair_use_findings["issue"].fillna("").astype(str)
)

# Basic cleaning: lowercase, collapse whitespace, trim
fair_use_findings["text"] = (
    fair_use_findings["text"]
    .str.lower()
    .str.replace(r"\s+", " ", regex=True)
    .str.strip()
)

# Save metadata feature variables (to prepare for later combination with SBERT)
metadata_features = fair_use_findings[['holding_word_count', 'tag_count', 'year_filled']].values

# Baseline Method (M3)

In Milestone 3, we extracted case topics using keyword frequency-based TF-IDF and NMF (Non-negative Matrix Factorization). This method had relatively low explanatory power (approximately 4.88% variance), limiting its ability to identify outliers in complex legal contexts. Below is the existing method, retained for comparison.

In [31]:
# TF-IDF & NMF
tfidf_vec = TfidfVectorizer(stop_words="english", min_df=2, max_df=0.95, ngram_range=(1, 2))
X_tfidf = tfidf_vec.fit_transform(fair_use_findings["text"])

nmf = NMF(n_components=10, random_state=42, init="nndsvda", max_iter=400)
nmf_doc_topic = nmf.fit_transform(X_tfidf)
fair_use_findings["nmf_dominant_topic"] = nmf_doc_topic.argmax(axis=1)

print("Baseline (M3) NMF executed for comparison.")

Baseline (M3) NMF executed for comparison.


# Method Upgrade (M4)

Milestone 4 introduces **SBERT (Sentence-BERT)** to understand the deep context of text. After combining the extracted 384-dimensional text embeddings with existing metadata, unlike M2, which relied on a single model (Isolation Forest), we additionally introduce LOF and a One-Class SVM to perform multi-validation.

In [32]:
# SBERT embeddings
sbert_model = SentenceTransformer('all-MiniLM-L6-v2')
text_embeddings = sbert_model.encode(fair_use_findings['text'].tolist(), show_progress_bar=True)

# Combining and Scaling Metadata and SBERT
X_combined_m4 = np.hstack([metadata_features, text_embeddings])
X_scaled_m4 = StandardScaler().fit_transform(X_combined_m4)

# Experiments with three outlier detection models (Contamination 0.05 standard)
contamination_rate = 0.05
models = {
    "IsolationForest": IsolationForest(contamination=contamination_rate, random_state=42),
    "LOF": LocalOutlierFactor(contamination=contamination_rate, novelty=True),
    "OneClassSVM": OneClassSVM(nu=contamination_rate, kernel="rbf")
}

for name, model in models.items():
    if name == "LOF":
        model.fit(X_scaled_m4)
        fair_use_findings[f'is_outlier_{name}'] = model.predict(X_scaled_m4)
    else:
        fair_use_findings[f'is_outlier_{name}'] = model.fit_predict(X_scaled_m4)

print("M4 Method Upgrade complete. 3 models applied.")

Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Batches:   0%|          | 0/8 [00:00<?, ?it/s]

M4 Method Upgrade complete. 3 models applied.


# Comparison & Evaluation

We use consensus between models to identify the most reliable red-flag cases. Furthermore, we verify the hypothesis established in M2—"outliers will be concentrated in certain courts (venues)"—using a chi-square test. Finally, we visualize the Risk Score in 3D using UMAP dimensionality reduction, establishing the foundation for an "Early Warning System" that enables legal teams to detect high-risk disputes early.

In [41]:
# Check the model consensus
outlier_cols = ['is_outlier_IsolationForest', 'is_outlier_LOF', 'is_outlier_OneClassSVM']
consensus_all = (fair_use_findings[outlier_cols] == -1).all(axis=1)
fair_use_findings['is_strong_outlier'] = consensus_all
print(f"Total strong Red-flag cases agreed by all 3 models: {consensus_all.sum()}")

# Identify Robust Red Flags (Agreed by at least 2 models)
fair_use_findings['consensus_score'] = (fair_use_findings[outlier_cols] == -1).sum(axis=1)
robust_red_flags = fair_use_findings[fair_use_findings['consensus_score'] >= 2].copy()
print(f"Number of robustly identified Red-flags (2+ models): {len(robust_red_flags)}")

# Top 10 robust cases
name_col = 'case' if 'case' in fair_use_findings.columns else 'case_number'
display(robust_red_flags[[name_col, 'outcome', 'consensus_score']].head(10))

Total strong Red-flag cases agreed by all 3 models: 2
Number of robustly identified Red-flags (2+ models): 12


,case_number,outcome,consensus_score
3,"No. 13-cv-1215 (TSC), 2022 U.S. Dist. LEXIS 60...",INDETERMINATE,2
10,"No. 18-cv-06637, 2021 U.S. Dist. LEXIS 184636 ...",FAIR_USE_FOUND,2
54,"291 F. Supp. 3d 422 (S.D.N.Y. Mar. 8, 2018)",FAIR_USE_FOUND,2
75,"No. 1.11-cv-1013-GTS-CFH (N.D.N.Y. Feb. 22, 2016)",INDETERMINATE,3
104,953 F. Supp. 2d 701 (N.D. Miss. 2013),FAIR_USE_FOUND,2
111,868 F. Supp. 2d 962 (C.D. Cal. 2012),FAIR_USE_FOUND,2
114,"No. 2:10-CV-1322 JCM (LRL) (D. Nev. Apr. 22, 2...",FAIR_USE_FOUND,2
116,607 F.3d 68 (2d Cir. 2010),INDETERMINATE,2
120,595 F.3d 1364 (Fed. Cir. 2010),FAIR_USE_NOT_FOUND,3
185,15 F.3d 559 (6th Cir. 1994),FAIR_USE_FOUND,2


In [55]:
# Statistical Verification: Venue vs. Outliers
contingency_table = pd.crosstab(fair_use_findings['court'], fair_use_findings['is_outlier_IsolationForest'])
chi2, p, dof, ex = chi2_contingency(contingency_table)

print(f"\nChi-Square P-value for Venue vs Outlier: {p:.4f}")
if p < 0.05:
    print("-> Conclusion: Significant relationship between Court (Venue) and Red-flag occurrence.")
else:
    print("-> Conclusion: No statistically significant relationship found.")


Chi-Square P-value for Venue vs Outlier: 0.0000
-> Conclusion: Significant relationship between Court (Venue) and Red-flag occurrence.


In [56]:
# Top 5 Most At-Risk Cases
top_risk_cases = fair_use_findings.sort_values(by='risk_score', ascending=False).head(5)
name_col = 'case' if 'case' in fair_use_findings.columns else 'case_number'

print("Top 5 Cases to highlight in the report as 'High Risk':")
display(top_risk_cases[[name_col, 'court', 'risk_score', 'outcome']])

Top 5 Cases to highlight in the report as 'High Risk':


,case_number,court,risk_score,outcome
120,595 F.3d 1364 (Fed. Cir. 2010),United States Court of Appeals for the Federal...,0.473834,FAIR_USE_NOT_FOUND
225,571 F. Supp. 144 (E.D. Pa. 1983),United States District Court for the Eastern D...,0.473346,FAIR_USE_NOT_FOUND
5,"Civ. Action No H-21-2612, 2022 U.S. Dist. LEXI...",United States District Court for the Southern ...,0.472998,FAIR_USE_NOT_FOUND
195,975 F. 2d 832 (Fed. Cir. 1992),United States Court of Appeals for the Federal...,0.472775,INDETERMINATE
75,"No. 1.11-cv-1013-GTS-CFH (N.D.N.Y. Feb. 22, 2016)",United States District Court for the Northern ...,0.472243,INDETERMINATE


In [57]:
# UMAP 3D
umap_3d = UMAP(n_components=3, random_state=42)
X_reduced = umap_3d.fit_transform(text_embeddings)

fair_use_findings['Outlier_Status'] = fair_use_findings['is_outlier_IsolationForest'].map({1: 'Normal', -1: 'Red-flag'})

fig = px.scatter_3d(
    fair_use_findings,
    x=X_reduced[:, 0],
    y=X_reduced[:, 1],
    z=X_reduced[:, 2],
    color='risk_score',
    symbol='Outlier_Status',
    hover_data=['case_number', 'court', 'outcome'],
    title="M4 Refinement: 3D Semantic Landscape & Risk Score (SBERT + UMAP)",
    color_continuous_scale='Reds', # Red represents high risk
    labels={'risk_score': 'Legal Risk Score'}
)

fig.update_layout(
    scene = dict(
        xaxis_title='Semantic Dimension 1',
        yaxis_title='Semantic Dimension 2',
        zaxis_title='Semantic Dimension 3'),
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.05,
        bgcolor="rgba(255, 255, 255, 0.5)"),
    coloraxis_colorbar=dict(
        title="Legal Risk Score",
        thicknessmode="pixels", thickness=20,
        lenmode="pixels", len=300,
        yanchor="middle", y=0.5,
        xanchor="left", x=1.05),
    margin=dict(l=0, r=100, b=0, t=40)
)

fig.show()

/usr/local/lib/python3.12/dist-packages/umap/umap_.py:1952: UserWarning:

n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.

